In [1]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.3 MB/s eta 0:00:00


In [2]:
!pip install streamlit pyngrok opencv-python-headless pillow tensorflow


In [3]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [5]:
# Install dependencies
!pip install tensorflow opencv-python-headless pillow gradio

# -------------------------------
# Imports
# -------------------------------
import gradio as gr
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------------------
# Load model (hardcoded path)
# -------------------------------
model_path = "/content/model.keras"  # adjust if needed
model = load_model(model_path, compile=False)

# -------------------------------
# Preprocessing function to remove black borders
# -------------------------------
def remove_black(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = gray > 10
    coords = np.argwhere(mask)
    if coords.size == 0:
        return img
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)
    cropped = img[y0:y1, x0:x1]
    gray_crop = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    mask_white = gray_crop <= 10
    cropped[mask_white] = [255, 255, 255]
    return cropped

# -------------------------------
# Preprocess image for the model
# -------------------------------
def preprocess_image(image: np.ndarray, target_size=(224,224)):
    # Apply remove_black first
    img_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Convert to BGR for remove_black
    processed_img_bgr = remove_black(img_bgr)
    processed_img_rgb = cv2.cvtColor(processed_img_bgr, cv2.COLOR_BGR2RGB) # Convert back to RGB

    # Resize and preprocess for the model
    resized_img = cv2.resize(processed_img_rgb, target_size)
    img_float = preprocess_input(resized_img.astype(np.float32))
    return img_float

# -------------------------------
# Prediction function
# -------------------------------
def predict_severity(image):
    img_float = preprocess_image(image)
    img_tensor = np.expand_dims(img_float, axis=0)
    preds = model.predict(img_tensor, verbose=0)
    labels = ['Mild', 'Medium', 'Severe']
    idx = np.argmax(preds[0])
    label = labels[idx]
    confidence = preds[0][idx] * 100
    return f"Predicted Severity: {label} ({confidence:.1f}%)"

# -------------------------------
# Gradio Interface
# -------------------------------
iface = gr.Interface(
    fn=predict_severity,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="🔬 Acne Severity Classifier",
    description="Upload a facial image to classify acne severity."
)

iface.launch(inline=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://45f90af2ee9743d3e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
